In [ ]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

model = "openai/gpt-5-mini"
temperature = 0

llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="medium",
    include_response_headers=True,
    use_responses_api=True,
)

In [ ]:
response = llm.invoke("Explain gradient descent")

2025-12-08 12:05:28,562 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
from langchain_playground.llm.openrouter import parse_invoke

parse_invoke(response, include_reasoning=True)

2025-12-08 12:04:32,376 - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.


("**Explaining gradient descent**\n\nThe user is asking for a general explanation of gradient descent, so I want to be concise yet thorough. I should consider including the intuition behind it, math, and its variants like batch, stochastic, and mini-batch approaches. I'll discuss the learning rate, convergence issues, and potential problems such as local minima or saddle points. It would be good to mention the algorithm steps and provide examples like linear regression, alongside relevant formulas in plain text. A simple 1D example or illustrating it with a quadratic function would also help make the concept accessible. I'll summarize with practical tips.**Structuring gradient descent explanation**\n\nI’m thinking about a structured approach for explaining gradient descent. I want to provide a short definition and intuitive idea followed by the mathematical formulation for minimizing a function f(θ) and the update rule. It’s essential to explain the learning rate and its consequences. 

In [1]:
"""OpenRouter LLM client initialization and configuration."""

import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

model = "openai/gpt-5-mini"
temperature = 0


llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="medium",  # Can be "minimal", "low", "medium", or "high"
    # use_responses_api=True,  # Required for reasoning_effort to work
)

In [2]:
from langchain_playground.llm.multimodal import MediaMessage

message = MediaMessage("input.png", description="Describe the weather in this image")
ai_msg = llm.invoke([message])
ai_msg

2025-12-08 14:45:37,477 - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.
2025-12-08 14:45:39,188 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='The image you provided is a screenshot of code (a JSON-like message object with a text prompt and an image URL). It does not show an outdoor scene, sky, or any photographic cues, so I can’t determine the weather from it.\n\nIf you want a weather description, please upload a photo that actually shows the scene (sky, landscape, street, etc.) or share metadata (EXIF) from the photo — I can then describe cloudiness, precipitation, lighting, wind clues, etc.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 652, 'total_tokens': 951, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 192, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 0.00075339, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 0.00

In [ ]:
from langchain.tools import tool

from langchain_playground.tools.youtube import youtube_loader


@tool
def scrape_youtube_video(youtube_url: str) -> str:
    """Scrape a YouTube video and return its transcript and metadata.

    This tool extracts the transcript, title, channel, duration, views, likes,
    and other metadata from a YouTube video URL.

    Args:
        youtube_url: The YouTube video URL to scrape (e.g., https://www.youtube.com/watch?v=VIDEO_ID)

    Returns:
        A formatted string containing video metadata and transcript
    """
    return youtube_loader(youtube_url)


tools = [scrape_youtube_video]

In [ ]:
from langchain.agents import create_agent

# Create a model for the agent (without structured output, as agents handle tool calling)
agent_llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="medium",
    # extra_body={"plugins": [{"id": "web"}]},
    use_responses_api=True,
)

# Create a simple agent with the YouTube scraper tool
youtube_agent = create_agent(
    model=agent_llm,
    tools=tools,
    # system_prompt="You are a helpful assistant that can analyze YouTube videos. When given a YouTube URL, use the scrape_youtube_video tool to get the video transcript and metadata, then provide insights or answer questions about the video content.",
)

In [ ]:
# Example: Scrape a YouTube video and ask questions about it
# Replace with your own YouTube URL
from langchain_core.messages import HumanMessage, SystemMessage

youtube_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Example URL

response = youtube_agent.invoke({"messages": [HumanMessage(content=youtube_url)]})

2025-12-08 12:11:51,274 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"
2025-12-08 12:11:59,745 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"


In [ ]:
parse_invoke(response["messages"][-1], include_reasoning=True)

('**Crafting a concise response**\n\nI\'ll reply to the user with something like: "I fetched the video. Metadata: ..." and then I’ll list the relevant details. I\'ll also mention that I retrieved the full transcript and lyrics. I think it\'s a good idea to offer the user some options for what they\'d like next. I\'ll keep the answer concise, maybe including the first lines of the lyrics, but that\'s not strictly necessary. Just need to keep it clear and to the point!',
 'I fetched that YouTube video. Summary of what I found:\n\n- Title: Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster)\n- Channel: Rick Astley\n- Duration: 3:34\n- Published: Oct 24, 2009\n- Views: 1,720,344,092\n- Likes: 18,668,354\n- Description: Official video + notes about the song/album, links to Rick Astley’s socials and streaming, plus full lyrics in the description.\n- Transcript: I retrieved the full transcript/lyrics (the familiar full lyrics are available).\n\nWhat would you like me to do wi